In [18]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [19]:
#movement of truck

def next_position(truck,time):
    x_curr = truck['x_curr']
    curr_dirn = truck['curr_dirn']
    speed = truck['speed']
    time = time
    return(x_curr + (curr_dirn*speed*time))

In [20]:
#forward moving truck arrival initiation

def for_truck_arr(trucks,time,position_of_junct):
    temp = {'x_curr':position_of_junct[0],'curr_dirn':1,'speed':1,'reverse_speed':1,'root_dirn':1,'arr_time':time,'depart_time':0,
        'for_dist':0,'back_dist':0}
    return trucks.append(temp)

In [21]:
#backward moving truck arrival initiation

def back_truck_arr(trucks,time,position_of_junct):
    temp = {'x_curr':position_of_junct[-1],'curr_dirn':-1,'speed':1,'reverse_speed':1,'root_dirn':-1,'arr_time':time,'depart_time':0,
        'for_dist':0,'back_dist':0}
    return trucks.append(temp)

In [22]:
#check if any of 2 trucks are near junctions, which means no need to consider collision

def not_near_junct(truck_1,truck_2,position_of_junct,time_delta):
    for i in range(0,len(position_of_junct)):
        if (abs(truck_1['x_curr'] - position_of_junct[i]) <= (truck_1['speed']*time_delta)) or (abs(truck_2['x_curr'] - position_of_junct[i]) <= (truck_2['speed']*time_delta)):
            return(0)
    
    return(1)

In [23]:
def dist_from_left_junc(x):
    
    for i in range(0,len(position_of_junct)):
        if x-position_of_junct[i] < 0:
            return(x-position_of_junct[i-1])
    
    return(0)

def dist_from_right_junc(x):
    
    for i in range(0,len(position_of_junct)):
        if x-position_of_junct[i] <= 0:
            return(abs(x-position_of_junct[i]))
    
    return(0)

In [24]:
#

def movement_desicion(trucks,collisions):
    
    if len(collisions) > 1:
        for i in range(0,len(collisions)-1):
            temp_coll = collisions[i]
            if len(temp_coll) > 1:                                     #merging collision pairs if multiple trucks on the same position
                for j in range(0,len(temp_coll)):
                    for k in range(i+1,len(collisions)):
                        if temp_coll[j] in collisions[k]:
                            collisions[i].extend(collisions[k])
                            collisions[k] = []

    for i in range(0,len(collisions)):
        if len(collisions[i]) <= 1:
            collisions[i] = []
 
    collisions = [x for x in collisions if x != []]                   #removing trucks with no collisions

    if len(collisions) > 0:
        for i in range(0,len(collisions)):                             #assigning direction to trucks at one point in order to clear traffic, acc to less reverse cumulative total dist to be travelled given which trucks move backward

            temp_coll = collisions[i]
            temp_for = 0
            temp_back = 0
            dist_from_prev_junc = dist_from_left_junc(trucks[temp_coll[0]]['x_curr'])
            dist_from_next_junc = dist_from_right_junc(trucks[temp_coll[0]]['x_curr'])

            for j in range(0,len(temp_coll)):
                if trucks[temp_coll[j]]['root_dirn'] == 1:
                    temp_for = temp_for + 1

                if trucks[temp_coll[j]]['root_dirn'] == -1:
                    temp_back = temp_back + 1

            if ((temp_for*dist_from_prev_junc) >= (temp_back*dist_from_next_junc)):
                temp_curr = 1

            else:
                temp_curr = -1

            for j in range(0,len(temp_coll)):
                trucks[temp_coll[j]]['curr_dirn'] = temp_curr

In [268]:
#initialization of params

time_delta = 0.0001                    #4th decimal place , will require this in rounding off to avoid floating point sum precision error
no_of_junctions = 18                   #including start and end node
#dist_bw_junct = [1]*no_of_junctions   #taken const and same
# mu=5
# sigma=2
# dist_bw_junct = np.random.normal(mu, sigma, 10)
# dist_bw_junct= [round(xyz,1) for xyz in dist_bw_junct]


# position_of_junct = [0]
# for i in range(1,no_of_junctions):
#     position_of_junct.append(position_of_junct[i-1]+dist_bw_junct[i-1])
mu=5
sigma=2
position_of_junct = np.random.normal(mu, sigma, no_of_junctions-2)
position_of_junct= [round(xyz,2) for xyz in position_of_junct]
print(position_of_junct)
position_of_junct.sort()
position_of_junct.append(10)
position_of_junct=[0]+position_of_junct
#junct_capacity = []
#junct_use_count = []

#trucks arrive at both ends following uniform dist
no_of_trucks_to_sim_each_dir = 5
sim_run_totaltime = 50
for_truck_arr_times = np.sort([round(num,2) for num in np.random.uniform(0,sim_run_totaltime/2,no_of_trucks_to_sim_each_dir)])
back_truck_arr_times = np.sort([round(num,2) for num in np.random.uniform(0,sim_run_totaltime/2,no_of_trucks_to_sim_each_dir)])

[6.73, 5.52, 6.26, 7.49, 9.06, 6.91, 2.49, 3.95, 7.09, 5.53, 4.69, 7.47, 7.24, 2.75, 8.18, 4.87]


In [269]:
position_of_junct

[0,
 2.49,
 2.75,
 3.95,
 4.69,
 4.87,
 5.52,
 5.53,
 6.26,
 6.73,
 6.91,
 7.09,
 7.24,
 7.47,
 7.49,
 8.18,
 9.06,
 10]

In [270]:
for_truck_arr_times

array([ 2.33,  3.66,  6.06,  6.88, 22.33])

In [271]:
back_truck_arr_times

array([ 9.2 ,  9.63, 19.58, 21.64, 22.22])

In [272]:
#simulation starts

trucks = []                                #list of trucks running
reached_trucks = []                        #list of trucks reached to their destination
t = 0                                      #global timeline

while t < sim_run_totaltime:                            #simulate for 1000 unit times or can change to condition when all trucks reach their detination 
    
    #print(t)
    
    if len(trucks) == 0 and t > max(max(for_truck_arr_times),max(back_truck_arr_times)):                   #stop simulation if all trucks reached
        print('simulation ends early')
        break
    
    if(t in for_truck_arr_times):          #trucks arrive at start node
        for_truck_arr(trucks,t,position_of_junct)
    if(t in  back_truck_arr_times):        #trucks arrive at end node 
        back_truck_arr(trucks,t,position_of_junct)
    
    t = round(t + time_delta,4)                     #time increment
    
    if len(trucks) > 0:                                    #all this calculations if any trucks running  
        for i in range(0,len(trucks)):                     #reseting truck movement direction ones it reaches junction after moving reverse dir
            if trucks[i]['x_curr'] in position_of_junct:
                trucks[i]['curr_dirn'] = trucks[i]['root_dirn']


        collisions = []                       #observing collisions over complete path
        temp_trucks = trucks
        for i in range(0,len(temp_trucks)):

            temp_coll = [i]
            temp_1 = temp_trucks[i]

            for j in range(i+1,len(temp_trucks)):

                temp_2 = temp_trucks[j]
                if not_near_junct(temp_1,temp_2,position_of_junct,time_delta):           #if not near junctions and unit dist close to each other moving in opp directions
                    if abs(temp_1['x_curr'] - temp_2['x_curr']) <= (temp_1['speed']*time_delta) and temp_1['curr_dirn'] != temp_2['curr_dirn']:
                        temp_coll.append(j)

            collisions.append(temp_coll)

        movement_desicion(trucks,collisions)              #change current directions of trucks acc to cumulative closeness to nearby nodes, in order to make decision which of the trucks move in reverse direction
        
        remove_trucks = []
        
        for i in range(0,len(trucks)):
            
                                        #stop trucks if reached destination, remove them from active running trucks
            
            if trucks[i]['root_dirn'] == 1:
                if (trucks[i]['x_curr'] >= position_of_junct[-1]):
                    trucks[i]['depart_time'] = t
                    reached_trucks.append(trucks[i])
                    remove_trucks.append(i)
                    #trucks.pop(i)
                    #trucks[i]['speed'] = 0
                    #trucks[i]['reverse_speed'] = 0 

            if trucks[i]['root_dirn'] == -1:
                if (trucks[i]['x_curr'] <= position_of_junct[0]):
                    trucks[i]['depart_time'] = t
                    reached_trucks.append(trucks[i])
                    remove_trucks.append(i)
                    #trucks.pop(i)
                    #trucks[i]['speed'] = 0
                    #trucks[i]['reverse_speed'] = 0 
        
        for k in remove_trucks:
            trucks.pop(k)
        
                                                                          #add forward or backward movement for trucks 
        if len(trucks) > 0:
            for i in range(0,len(trucks)):
                if trucks[i]['curr_dirn'] == trucks[i]['root_dirn']:
                    trucks[i]['for_dist'] = trucks[i]['for_dist'] + trucks[i]['speed']*time_delta

                else:
                    trucks[i]['back_dist'] = trucks[i]['back_dist'] + trucks[i]['speed']*time_delta


                trucks[i]['x_curr'] = round(next_position(trucks[i],time_delta),4)     #move truck to next position accordingly

simulation ends early


In [273]:
t

32.4801

In [274]:
trucks

[]

In [275]:
reached_trucks

[{'x_curr': 10.0,
  'curr_dirn': 1,
  'speed': 1,
  'reverse_speed': 1,
  'root_dirn': 1,
  'arr_time': 2.33,
  'depart_time': 13.2701,
  'for_dist': 10.469999999988937,
  'back_dist': 0.46999999999996456},
 {'x_curr': 10.0,
  'curr_dirn': 1,
  'speed': 1,
  'reverse_speed': 1,
  'root_dirn': 1,
  'arr_time': 3.66,
  'depart_time': 14.6501,
  'for_dist': 10.49499999998888,
  'back_dist': 0.4949999999999618},
 {'x_curr': 10.0,
  'curr_dirn': 1,
  'speed': 1,
  'reverse_speed': 1,
  'root_dirn': 1,
  'arr_time': 6.06,
  'depart_time': 17.1101,
  'for_dist': 10.52499999998881,
  'back_dist': 0.5249999999999585},
 {'x_curr': 10.0001,
  'curr_dirn': 1,
  'speed': 1,
  'reverse_speed': 1,
  'root_dirn': 1,
  'arr_time': 6.88,
  'depart_time': 17.1102,
  'for_dist': 10.115099999989765,
  'back_dist': 0.11500000000000227},
 {'x_curr': 10.0001,
  'curr_dirn': 1,
  'speed': 1,
  'reverse_speed': 1,
  'root_dirn': 1,
  'arr_time': 6.88,
  'depart_time': 17.1102,
  'for_dist': 10.115099999989765,


In [276]:
tot_back_dist=0
tot_forw_dist=0
for obj in reached_trucks:
    total_dist=obj['for_dist']+obj['back_dist']
    obj['total_dist']=total_dist
    total_time=obj['depart_time']-obj['arr_time']
    obj['total_time']=total_time
    tot_back_dist=tot_back_dist+obj['back_dist']
    tot_forw_dist=tot_forw_dist+obj['for_dist']
total_dist=0
total_time=0

for truck in reached_trucks:
    total_dist=total_dist+truck['total_dist']
    total_time=total_time+truck['total_time']
avg_dist=total_dist/len(reached_trucks)
avg_time=total_time/len(reached_trucks)
avg_back_dist=tot_back_dist/len(reached_trucks)
avg_f=tot_forw_dist/len(reached_trucks)

print("Average Time: ", avg_time)
print("Average Forw Distance",avg_f)
print("Average Backward Dist",avg_back_dist )
print("Average Distance", avg_dist)


Average Time:  10.431119999999998
Average Forw Distance 10.215519999989532
Average Backward Dist 0.2154999999999871
Average Distance 10.431019999989518


In [277]:
print("Total Time: ", total_time)
print("For Distance",tot_forw_dist)
print("Backward:",tot_back_dist )
print("Total Distance", total_dist)


Total Time:  104.31119999999999
For Distance 102.15519999989532
Backward: 2.154999999999871
Total Distance 104.31019999989518
